In [1]:
import string
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Enzo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Enzo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Enzo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Consigna 1

Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

### Carga de datos y vectorización

In [2]:
# Carga del dataset 20newsgroups.
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

# Vectorización con TF-IDF del set de datos de entrenamiento.
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data) # Matriz documento-término
y_train = newsgroups_train.target

docs_count = X_train.shape[0]
print(f'El corpus consta de {docs_count} documentos\n')

print(f'Las clases en las que los documentos son clasificados son\n{newsgroups_test.target_names}\n')

# Porcentaje de documentos para cada clase.
for t in np.unique(newsgroups_test.target):
    print(f'El {np.count_nonzero(newsgroups_test.target == t) / docs_count:.2%} de los datos pertenece a la clase {newsgroups_test.target_names[t]}')

print('\nAlgunos de los términos del vectorizador:')
for term in np.sort(list(tfidfvect.vocabulary_.keys()))[20000:20020]:
    print(term)

El corpus consta de 11314 documentos

Las clases en las que los documentos son clasificados son
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']

El 2.82% de los datos pertenece a la clase alt.atheism
El 3.44% de los datos pertenece a la clase comp.graphics
El 3.48% de los datos pertenece a la clase comp.os.ms-windows.misc
El 3.46% de los datos pertenece a la clase comp.sys.ibm.pc.hardware
El 3.40% de los datos pertenece a la clase comp.sys.mac.hardware
El 3.49% de los datos pertenece a la clase comp.windows.x
El 3.45% de los datos pertenece a la clase misc.forsale
El 3.50% de los datos pertenece a la clase rec.autos
El 3.52% de los datos pertenec

### Vectorizar 5 documentos y estudiar similitud

In [3]:
# Seleccionar al azar los índices de 5 documentos procedentes del set de entrenamiento.
# Para cada uno de ellos, obtener los índices de los 5 documentos más similares y almacenar
# en un diccionario.
indices = np.random.randint(0, X_train.shape[0] - 1, size=5)
doc_to_similars = {}
for i in indices:
    cos_sim = cosine_similarity(X_train[i], X_train)[0]
    most_sim = np.argsort(cos_sim)[::-1][1:6]
    doc_to_similars[i] = most_sim
    print(f'Los 5 documentos más similares al documento en índice {i} son: {most_sim}')

Los 5 documentos más similares al documento en índice 5287 son: [2256 6243 4603 5898 2825]
Los 5 documentos más similares al documento en índice 7988 son: [ 8422 10406  4473 10505  3318]
Los 5 documentos más similares al documento en índice 4357 son: [9623 6635 6894 7162 3141]
Los 5 documentos más similares al documento en índice 5327 son: [7286 5522 1292 5829 7963]
Los 5 documentos más similares al documento en índice 1831 son: [3008 4496 8744 7553  562]


#### Analizar documento 1

In [4]:
# El documento 1 es:
doc_idx = 0
print(newsgroups_train.data[indices[doc_idx]])

# Su target es:
target_doc = newsgroups_train.target_names[y_train[indices[doc_idx]]]
print(f'\nSu target es {target_doc}')



I don't know, but I'm as willing to speculate as anyone.

Several people have suggested that the chips use public-key cryptography.
Another possibility is to use Diffie-Hellman key exchange, or some other
algorithm which has a similar effect.  DH allows both ends to agree on
a session key which they use with symmetric cryptography (something
like DES) for the encryption.

How could the "back door" keys work in this system?  I can see a few
possibilities.  One is that the DH-like algorithm has the property that
the session key can be deduced by an eavesdropper who knows the back door
keys for one (or possibly both) communicants.  Perhaps the random numbers
used in the DH are seeded by the back door key, or some such.

Another possibility, as was suggested here earlier, is that the chips
simply broadcast the session key, encrypted with the chip's own
internal secret key.  In this system the back door keys are secret keys
usable for decrypting this session key broadcast.  Actually the c

In [5]:
# Los 5 más similares al documento 1 son:
targets_accum = 0
for i in range(5):
    j = doc_to_similars[indices[doc_idx]][i]
    print(newsgroups_train.data[j])
    target_similar = newsgroups_train.target_names[y_train[j]]
    targets_accum += 1 if target_doc == target_similar else 0
    print(f'\nSu target es {target_similar}')
    print('\n====================================================\n')

print(f'\nEl {100 * targets_accum / 5}% dos los 5 más similares comparten el mismo target.')

 
   Not necessarily.  I've been thinking about this, and if this chip/scheme
is to provide any real security, there must be some sort of key exchange,
either using a public-key encryption scheme, or using a key exchange scheme
like Diffie-Hellman.  If there's an out-of-band transmission of a shared
session key, then what protects that band from eavesdropping?  If the phone
company or some other online central authority generates a session key and
sends it to both users, then what's the point of going to the trouble of
having some complicated key-depositories?  Just ask the phone company for
a copy of the session key for each call.
 
   Now, it's probably not practical for each user to keep an online copy of
every public key used by anyone anywhere, right?  So, probably, there will
be some way of getting these keys verified.  This might be a digitally-
signed (by the chip manufacturer) copy of the public key in this unit,
stored by this unit.  It might also be an online directory with 

#### Analizar documento 2

In [6]:
# El documento 2 es:
doc_idx = 1
print(newsgroups_train.data[indices[doc_idx]])

# Su target es:
target_doc = newsgroups_train.target_names[y_train[indices[doc_idx]]]
print(f'\nSu target es {target_doc}')



In the latest case in Denver, they were giving away tickets to a Denver
Nuggets basketball game. 

How traceable is a money order?  (I don't know. Haven't used one in 20 years)

Is that even an issue if the weapons aren't checked for being stolen?

Su target es talk.politics.guns


In [7]:
# Los 5 más similares al documento 2 son:
targets_accum = 0
for i in range(5):
    j = doc_to_similars[indices[doc_idx]][i]
    print(newsgroups_train.data[j])
    target_similar = newsgroups_train.target_names[y_train[j]]
    targets_accum += 1 if target_doc == target_similar else 0
    print(f'\nSu target es {target_similar}')
    print('\n====================================================\n')

print(f'\nEl {100 * targets_accum / 5}% dos los 5 más similares comparten el mismo target.')

Re: More on Gun Buybacks

The Denver buy back, trading guns for Denver Nuggets tickets was pretty much
a bust. Very few guns were turned in. The news tried to hype it but 
when the best they could do was ".... including a loaded .38..." well,
you get the picture.

A side note- the news also reported that the guns would be checked for
whether or not they were stolen. STOLEN GUNS WILL BE RETURNED TO THEIR
OWNERS!!!!! (They say)

(Does this have anything to do with the rally on the Capital steps yesterday
 in support of the RKBA????)

Even the rally made the 5 pm news on 3 channels :-)


Su target es talk.politics.guns




       Money orders operate pretty much like checks, with both parties being
supposed to sign them.  I assume you'd have to show the buy-back people
an ID, and you'd then have a money order made out to that ID.  

       As far as traceable as a practical matter, I don't know, it would
depend on whether they bother to computerize who the recipient's name is
on the money

#### Analizar documento 3

In [8]:
# El documento 3 es:
doc_idx = 2
print(newsgroups_train.data[indices[doc_idx]])

# Su target es:
target_doc = newsgroups_train.target_names[y_train[indices[doc_idx]]]
print(f'\nSu target es {target_doc}')


They do. According the the Los Angeles Police Department, illegal
manufacture is one the three primary sources of machine guns and
submachine guns used in crimes (sumggling and theft from the
police and military being the other two.) Washington D.C. police
have stated that 40% (If I'm remembering the figure correctly) of
the guns they conficate were illegally built.


It takes about 6 hours and a few tools to make one (at least one
of reasonable quality). Unless the drug dealer enjoyes messing
around on a lathe (say, as a hobby), he's going to have to 
pay someone anyway. Materials plus six hours of a machinist's
time for something legal would run about $100. The blackmarket
prices for guns are usually in the $50 to $200 range (at least
those few I've seen cited in newspaper articles were...)


Washington D.C. has a total ban on handguns and prohibits assembled
rifles within city limits. It's homicide rate is almost ten
times the national average. It is also illegal for a D.C.
residen

In [9]:
# Los 5 más similares al documento 3 son:
targets_accum = 0
for i in range(5):
    j = doc_to_similars[indices[doc_idx]][i]
    print(newsgroups_train.data[j])
    target_similar = newsgroups_train.target_names[y_train[j]]
    targets_accum += 1 if target_doc == target_similar else 0
    print(f'\nSu target es {target_similar}')
    print('\n====================================================\n')

print(f'\nEl {100 * targets_accum / 5}% dos los 5 más similares comparten el mismo target.')

Accounts of Anti-Armenian Human Right Violations in Azerbaijan #012
                 Prelude to Current Events in Nagorno-Karabakh

        +---------------------------------------------------------+
        |                                                         |
        |  I saw a naked girl with her hair down. They were       |
        |  dragging her. She kept falling because they were       |
        |  pushing her and kicking her. She fell down, it was     |
        |  muddy there, and later other witnesses who saw it from |
        |  their balconies told us, they seized her by the hair   |
        |  and dragged her a couple of blocks, as far as the      |
        |  mortgage bank, that's a good block and a half or two   |
        |  from here. I know this for sure because I saw it       |
        |  myself.                                                |
        |                                                         |
        +-------------------------------------------

#### Analizar documento 4

In [10]:
# El documento 4 es:
doc_idx = 3
print(newsgroups_train.data[indices[doc_idx]])

# Su target es:
target_doc = newsgroups_train.target_names[y_train[indices[doc_idx]]]
print(f'\nSu target es {target_doc}')

Piper lived in my town (Williamsport, PA) when he killed himself.  It
was in the early '60's.  He had had more than a few books published by
that time, but he was down on his luck financially.  Rumor was that he
was hunting urban pigeons with birdshot for food.  He viewed himself as
a resourceful man, and (IMO) decided to check out gracefully if he
couldn't support himself.  The worst part is that John Campbell, the
long-time editor of Astounding/Analog SF magazine had cut a check for
Piper's most recent story, and said check was in the mail.  If Campbell
had known Piper's straits, I'm sure he would have phoned to say hang on.
Campbell was like that.

I wish it had happened differently.  I always enjoyed Piper's stuff.

Su target es sci.space


In [11]:
# Los 5 más similares al documento 4 son:
targets_accum = 0
for i in range(5):
    j = doc_to_similars[indices[doc_idx]][i]
    print(newsgroups_train.data[j])
    target_similar = newsgroups_train.target_names[y_train[j]]
    targets_accum += 1 if target_doc == target_similar else 0
    print(f'\nSu target es {target_similar}')
    print('\n====================================================\n')

print(f'\nEl {100 * targets_accum / 5}% dos los 5 más similares comparten el mismo target.')

Accounts of Anti-Armenian Human Right Violations in Azerbaijan #008 Part A
                 Prelude to Current Events in Nagorno-Karabakh
	
				(Part A of #008)

      +------------------------------------------------------------------+
      |                                                                  |
      | "Oh, yes, I just remembered. While they were raping me they      |
      |  repeated quite frequently, "Let the Armenian women have babies  |
      |  for us, Muslim babies, let them bear Azerbaijanis for the       |
      |  struggle against the Armenians." Then they said, "Those         |
      |  Muslims can carry on our holy cause. Heroes!" They repeated     |
      |  it very often."                                                 |
      |                                                                  |
      +------------------------------------------------------------------+

DEPOSITION OF LYUDMILA GRIGOREVNA M.

   Born 1959
   Teacher
   Sumgait Secondary Scho

#### Analizar documento 5

In [12]:
# El documento 5 es:
doc_idx = 4
print(newsgroups_train.data[indices[doc_idx]])

# Su target es:
target_doc = newsgroups_train.target_names[y_train[indices[doc_idx]]]
print(f'\nSu target es {target_doc}')

I am looking for EISA or VESA local bus graphic cards that support at least 
1024x786x24 resolution.  I know Matrox has one, but it is very expensive.  All the
other cards I know of, that support that resoultion, are striaght ISA. 

Also are there any X servers for a unix PC that support 24 bits?

Su target es comp.graphics


In [13]:
# Los 5 más similares al documento 5 son:
targets_accum = 0
for i in range(5):
    j = doc_to_similars[indices[doc_idx]][i]
    print(newsgroups_train.data[j])
    target_similar = newsgroups_train.target_names[y_train[j]]
    targets_accum += 1 if target_doc == target_similar else 0
    print(f'\nSu target es {target_similar}')
    print('\n====================================================\n')

print(f'\nEl {100 * targets_accum / 5}% dos los 5 más similares comparten el mismo target.')

Does XFree86 support any EISA video cards under Dell 2.2?

Su target es comp.windows.x




That's nonsense!!  You can use ISA cards in an EISA-system without problem
and at the same speed as in an ISA system!!

Su target es comp.sys.ibm.pc.hardware


To those interested in the new ATI Ultra Cards:

I had posted some questions regarding the new ATI Ultra Pro cards and
had asked confirmation of some opinions.

This message is a summary of the responses. Thanks to all of you that
replied.



a) The higher memory limits apply to ISA cards only, as far as I know.  The VLB
   and EISA version should have no problems.
 
b) I'm pretty sure from my experience that the ISA version doesn't
   work in systems with over 16M Ram.  There is supposed to be way
   of switching the "memory aperture" feature off to prevent this,
   but apparently it doesn't work.  I posted some "help me" messages
   on the net and people indicated that the EISA card didn't have this
   problem.

c) FALSE

d) The VLB card

### Comentarios

Se tomaron los 5 documentos más similares a cada uno de 5 documentos elegidos al azar. Se usó como métrica la distancia del coseno y a TF-IDF como técnica de vectorización. Para cada uno de los documentos analizados se calculó el porcentaje de los 5 más similares que poseen el mismo target que el documento analizado. En sucesivas pruebas se observó que a veces dicho porcentaje fue inferior al 50%, lo que sugiere que la técnica de vectorización empleada podría ser mejorada.

## Consigna 2

Entrenar modelos de clasificación Naive Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naive Bayes Multinomial
y ComplementNB.

### Naive Bayes y Complement Naive Bayes usando la vectorización obtenida hasta aquí y parámetros por defecto

In [14]:
# Instanciar y fitear un multinomial Naive Bayes classifier.
mult_nb = MultinomialNB()
mult_nb.fit(X_train, y_train)

# Vectorizar set de datos de testing y realizar predict.
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  mult_nb.predict(X_test)

# Calcular F1-score macro.
print(f'F1 score macro con multinomial Naive Bayes:', f1_score(y_test, y_pred, average='macro'))

# Instanciar y fitear un complement Naive Bayes classifier.
compl_nb = ComplementNB()
compl_nb.fit(X_train, y_train)

# Realizar predict.
y_pred = compl_nb.predict(X_test)

# Calcular F1-score macro.
print(f'F1 score macro con complement Naive Bayes:', f1_score(y_test, y_pred, average='macro'))

F1 score macro con multinomial Naive Bayes: 0.5854345727938506
F1 score macro con complement Naive Bayes: 0.692953349950875


### Complement Naive Bayes usando stopwords

In [15]:
# Volver a instanciar el vectorizador, esta vez usando stop words del inglés.
tfidfvect = TfidfVectorizer(stop_words='english')
X_train = tfidfvect.fit_transform(newsgroups_train.data)

X_test = tfidfvect.transform(newsgroups_test.data)

# Instanciar y fitear un complement Naive Bayes classifier.
compl_nb = ComplementNB()
compl_nb.fit(X_train, y_train)

# Realizar predict.
y_pred = compl_nb.predict(X_test)

print(f'F1 score macro con complement Naive Bayes y vectorización con stopwords es :', f1_score(y_test, y_pred, average='macro'))

F1 score macro con complement Naive Bayes y vectorización con stopwords es : 0.6936107849650025


### Complement Naive Bayes usando preprocesamiento con NLTK

In [16]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def nltk_preprocess(document):
    '''
    Aplica tokenization, filta stopwords en inglés, filtra signos de puntuación y aplica lemmatization
    a un documento recibido como parámetro. Retorna el documento preprocesado.
    '''
    tokens = word_tokenize(document.lower())
    tokens = [token for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in string.punctuation]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

# Preprocesar todos los documentos.
newsgroups_train.data = [nltk_preprocess(doc) for doc in newsgroups_train.data]
newsgroups_test.data = [nltk_preprocess(doc) for doc in newsgroups_test.data]

# Volver a instanciar el vectorizador.
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)

X_test = tfidfvect.transform(newsgroups_test.data)

# Instanciar y fitear un complement Naive Bayes classifier.
compl_nb = ComplementNB()
compl_nb.fit(X_train, y_train)

# Realizar predict.
y_pred = compl_nb.predict(X_test)

print(f'F1 score macro con complement Naive Bayes y preprocesamiento con NLTK:', f1_score(y_test, y_pred, average='macro'))

F1 score macro con complement Naive Bayes y preprocesamiento con NLTK: 0.6793289730320086


## Consigna 3

Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

In [17]:
# Transponer X_train para obtener la matriz de términos a documentos. Cada fila puede ser interpretada como una vectorización de un término.
X_train_t = X_train.T
X_train_t.shape

# Contruir diccionario que mapea índices con términos.
idx_to_term = { v: k for k, v in tfidfvect.vocabulary_.items() }

# Seleccionar al azar los índices de 5 términos procedentes del set de entrenamiento.
# Para cada uno de ellos, obtener los índices de los 5 términos más similares y almacenar
# en un diccionario.
indices = np.random.randint(0, X_train_t.shape[0] - 1, size=5)
term_to_similars = {}
for i in indices:
    cos_sim = cosine_similarity(X_train_t[i], X_train_t)[0]
    most_sim = np.argsort(cos_sim)[::-1][1:6]
    term_to_similars[i] = most_sim
    print(f'Los 5 términos más similares al término en índice {i} son: {most_sim}')

Los 5 términos más similares al término en índice 53612 son: [ 2165 38199 16839 43790  5232]
Los 5 términos más similares al término en índice 26852 son: [33221 27893 18312 33776 33772]
Los 5 términos más similares al término en índice 20225 son: [ 3995 36716 11033  8477 23332]
Los 5 términos más similares al término en índice 41935 son: [29797 14630 49149 41652 28629]
Los 5 términos más similares al término en índice 10588 son: [43000 43975 49907 20195 20196]


#### Analizar término 1

In [18]:
# El término 1 es:
term_idx = 0
print(f'El término en índice {indices[term_idx]} es {idx_to_term[indices[term_idx]]}')

El término en índice 53612 es wondering


In [19]:
# Los 5 más similares al término 1 son:
for i in range(5):
    j = term_to_similars[indices[term_idx]][i]
    print(f'Índice: {j}. Term: {idx_to_term[j]}')
    print('\n====================================================\n')

Índice: 2165. Term: anyone


Índice: 38199. Term: prophetic


Índice: 16839. Term: fenway


Índice: 43790. Term: shirley


Índice: 5232. Term: bloated




#### Analizar término 2

In [20]:
# El término 2 es:
term_idx = 1
print(f'El término en índice {indices[term_idx]} es {idx_to_term[indices[term_idx]]}')

El término en índice 26852 es landon


In [21]:
# Los 5 más similares al término 2 son:
for i in range(5):
    j = term_to_similars[indices[term_idx]][i]
    print(f'Índice: {j}. Term: {idx_to_term[j]}')
    print('\n====================================================\n')

Índice: 33221. Term: noll


Índice: 27893. Term: llat


Índice: 18312. Term: fuscare


Índice: 33776. Term: obfuscatory


Índice: 33772. Term: obfuscare




#### Analizar término 3

In [22]:
# El término 3 es:
term_idx = 2
print(f'El término en índice {indices[term_idx]} es {idx_to_term[indices[term_idx]]}')

El término en índice 20225 es habit


In [23]:
# Los 5 más similares al término 3 son:
for i in range(5):
    j = term_to_similars[indices[term_idx]][i]
    print(f'Índice: {j}. Term: {idx_to_term[j]}')
    print('\n====================================================\n')

Índice: 3995. Term: bareback


Índice: 36716. Term: placate


Índice: 11033. Term: dangles


Índice: 8477. Term: coaster


Índice: 23332. Term: inferring




#### Analizar término 4

In [24]:
# El término 4 es:
term_idx = 3
print(f'El término en índice {indices[term_idx]} es {idx_to_term[indices[term_idx]]}')

El término en índice 41935 es rubbing


In [25]:
# Los 5 más similares al término 4 son:
for i in range(5):
    j = term_to_similars[indices[term_idx]][i]
    print(f'Índice: {j}. Term: {idx_to_term[j]}')
    print('\n====================================================\n')

Índice: 29797. Term: meguire


Índice: 14630. Term: elixir


Índice: 49149. Term: trackball


Índice: 41652. Term: roller


Índice: 28629. Term: magic




#### Analizar término 5

In [26]:
# El término 5 es:
term_idx = 4
print(f'El término en índice {indices[term_idx]} es {idx_to_term[indices[term_idx]]}')

El término en índice 10588 es cscx


In [27]:
# Los 5 más similares al término 5 son:
for i in range(5):
    j = term_to_similars[indices[term_idx]][i]
    print(f'Índice: {j}. Term: {idx_to_term[j]}')
    print('\n====================================================\n')

Índice: 43000. Term: scx


Índice: 43975. Term: shzrlk


Índice: 49907. Term: txrbc


Índice: 20195. Term: gytz


Índice: 20196. Term: gyx




### Comentarios

Se tomaron los 5 términos más similares a cada uno de 5 términos elegidos al azar. Se usó como métrica la distancia del coseno y a TF-IDF como técnica de vectorización. En sucesivas pruebas se observó que no existe semejanza semántica entre los términos comparados. Esto es esperable dada la técnica de vectorización empleada. TF-IDF no capta el significado semántico de las palabras y no considera el contexto en el que las mismas figuran. Palabras como "silla" y "asiento", por ejemplo, podrían tener vectores no cercanos en función de las diferentes frencuencias con las que estas palabras puedan aparecer en un determinado corpus. Distinto habría sido el resultado de haber usado word embeddings sobre un corpus suficientemente rico.